In [ ]:
from shapely.geometry import MultiPolygon, Polygon, mapping, shape
import json
from shapely.geometry import shape, mapping
from shapely.ops import unary_union
import os
import csv
from shapely.ops import transform
from pyproj import Transformer
import json


In [ ]:
file = "../rawData/countries.geoJson"
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)

with open(file, 'r') as f:
    data = json.load(f)

## Missing Iso

In [ ]:
# list all names if 'ISO3166-1-Alpha-3' == -99
missing_names = [
    feature['properties']['name'] for feature in data['features']
    if feature['properties']['ISO3166-1-Alpha-3'] == '-99'
]   
print("Missing ISO codes for countries:")
for name in missing_names:
    print(f"{name}")

In [ ]:
missingIso = {
    "France": "FRA",
    "Norway": "NOR",
    "Somaliland": "SOM",
    "Kosovo": "KOS",
    "Dhekelia Sovereign Base Area": "SBA"
}

# replace -99 with corresponding name
for feature in data['features']:
    if feature['properties']['name'] in missingIso:
        feature['properties']['ISO3166-1-Alpha-3'] = missingIso[feature['properties']['name']]

## Process Data

In [ ]:

tolerance = 0
tol_str = str(tolerance).replace('.', '_')
normalize = True
scale_factor = 100000

crs = "EPSG:3857"
if crs == "EPSG:3857":
    crs_path = "WebMercator"
else:
    crs_path = "WGS84"

# output_dir = "../processedData/test"
output_dir = f'../processedData/{crs_path}/tol{tol_str}_scale{scale_factor}'

def scale_coords(x, y, z=None):
    return x / scale_factor, y / scale_factor

# --- LOAD DATA ---
with open(file, 'r') as f:
    data = json.load(f)

# --- TRANSFORMER (WGS84 -> Web Mercator) ---
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)

for idx in range(len(data['features'])):
# for idx in range(3):
    # Exclude Antarctica
    if data['features'][idx]['properties']['ISO3166-1-Alpha-3'] == "ATA":
        continue

        # Process the feature
    feature = data['features'][idx]
    iso3 = feature['properties']['ISO3166-1-Alpha-3']
    geometry = shape(feature['geometry'])
    geometry = transform(transformer.transform, geometry)
    geometry = transform(scale_coords, geometry)               # Apply scaling
    geometry = geometry.simplify(tolerance, preserve_topology=True)

    # --- Make polygons iterable ---
    if isinstance(geometry, Polygon):
        polygons = [geometry]
    elif isinstance(geometry, MultiPolygon):
        polygons = list(geometry.geoms)
    else:
        raise TypeError(f"Unsupported geometry type: {geometry.geom_type}")

    outline_coords = []

    for polygon in polygons:
        outline = polygon.boundary
        lines = outline.geoms if outline.geom_type == 'MultiLineString' else [outline]
        for line in lines:
            coords = list(line.coords)
            outline_coords.append(coords)
    print(f"[✓] Processed {iso3} with {len(outline_coords)} outlines")
   

    os.makedirs(output_dir, exist_ok=True)
    out_path = os.path.join(output_dir, f"{iso3}.json")

    with open(out_path, 'w') as f_out:
        json.dump({"coordinates": outline_coords}, f_out)

    print(f"[✓] Saved {iso3} to {out_path}")

## Test plot

In [ ]:
# plot all json 
path = "../processedData/WebMercator/tol0"

import matplotlib.pyplot as plt 
import json
import os
color = ['red', 'green', 'blue', 'orange', 'purple', 'brown', 'pink', 'gray']

# plot all files in path 
files = [f for f in os.listdir(path) if f.endswith('.json')]
files
for i, file in enumerate(files):
    print(f"Processing {file}...")
    with open(os.path.join(path, file), 'r') as f:
        data = json.load(f)
    coords = data['coordinates']
    for coord in coords:
        line = LineString(coord)
        x, y = line.xy
        plt.plot(x, y, color=color[i % len(color)], linewidth=1)


